In [17]:
import pandas as pd
from pprint import pprint

ARQUIVO_DADOS = 'dados_oceans_status.csv'

In [18]:
df_oceans = pd.read_csv(ARQUIVO_DADOS)
df_oceans.columns

Index(['tick', 'P_L', 'ano', 'margem_liq', 'divida_liq_EBITDA', 'liq_corrente',
       'liq_imediata', 'LPA', 'VPA', 'P_VP', 'P_EBIT', 'P_EBITDA', 'P_ativos',
       'margem_EBITDA', 'PSR', 'ROE', 'ROA', 'ROIC', 'EV_EBIT', 'EV_EBITDA',
       'DY', 'preco', 'margem_bruta', 'margem_EBIT', 'divida_liq_EBIT',
       'divida_liq_patrimonio', 'P_cap_giro', 'P_cirulante_liq',
       'patrimonio_ativos', 'passivos_ativos', 'giro_ativos', 'receitas_5anos',
       'lucros_5anos', 'liq_media_diaria'],
      dtype='object')

In [19]:
def somar_posicoes(df_rank, list_indicadores):
    linhas_posi = []
    for tick, linha in df_rank.iterrows():
        soma_posi = 0
        for i in range(1, len(linha), 1):
            soma_posi += linha[i]
        linhas_posi.append([tick, soma_posi])
    
    df_posi = pd.DataFrame(linhas_posi, columns=['tick', 'soma_posi'])
    df_posi.set_index('tick', inplace=True)
    return df_rank.join(df_posi).copy()

In [20]:
def calcular_ranking(df_completo, list_indicadores, ano):
    '''
    DataFrame - dados completos
    list_indicadores - [['TICKER', bool (O primeiro lugar é o menor valor?)], [Ticker, bool], ...]
    '''
    df_rankings = df_completo[df_completo['ano'] == ano][['tick', 'ano']].copy()
    df_rankings.set_index('tick', inplace=True)

    for indicador, asc in list_indicadores:
        if(not indicador == 'liq_media_diaria'):
            df_indicador = df_completo[df_completo['ano'] == ano][['tick', indicador]].copy()
            df_indicador.set_index('tick', inplace=True)

            df_indicador['rank_'+indicador] = df_indicador[indicador].rank(ascending=asc, na_option='bottom')
            df_indicador.drop(indicador, axis=1, inplace=True)

            df_rankings = df_rankings.join(df_indicador)
    
    df_rankings = somar_posicoes(df_rankings, list_indicadores)
    return df_rankings.sort_values('soma_posi').copy()

In [21]:
df_oceans.columns

Index(['tick', 'P_L', 'ano', 'margem_liq', 'divida_liq_EBITDA', 'liq_corrente',
       'liq_imediata', 'LPA', 'VPA', 'P_VP', 'P_EBIT', 'P_EBITDA', 'P_ativos',
       'margem_EBITDA', 'PSR', 'ROE', 'ROA', 'ROIC', 'EV_EBIT', 'EV_EBITDA',
       'DY', 'preco', 'margem_bruta', 'margem_EBIT', 'divida_liq_EBIT',
       'divida_liq_patrimonio', 'P_cap_giro', 'P_cirulante_liq',
       'patrimonio_ativos', 'passivos_ativos', 'giro_ativos', 'receitas_5anos',
       'lucros_5anos', 'liq_media_diaria'],
      dtype='object')

In [22]:
def analisar_indicadores(lista_indicadores, ano):
    ''' A partir de uma lista de indicadores e um ano, retorna uma dataframe com o ranking de cada empresa
    recebe:
    dict - 'indicador': [min, max]
    int - ano

    retorna
    DataFrame - tabela com: tick, posicao geral, posicao normalizada, posicao no indicador 1, 2, 3, ..., n
    '''
    DICT_ASC = {
        'divida_liq_EBITDA': ['divida_liq_EBITDA', True],
        'DY': ['DY', False],
        'ROA': ['ROA', False],
        'ROE': ['ROE', False],
        'ROIC': ['ROIC', False],
        'P_L': ['P_L', True],
        'P_VP': ['P_VP', True],
        'LPA': ['LPA', False],
        'VPA': ['VPA', True],
        'PSR': ['PSR', False],
        'P_EBIT': ['P_EBIT', True],
        'P_EBITDA': ['P_EBITDA', True],
        'P_ativos': ['P_ativos', True],
        'P_cap_giro': ['P_cap_giro', False],
        'P_cirulante_liq': ['P_cirulante_liq', False],
        'margem_EBITDA': ['margem_EBITDA', False],
        'margem_EBIT': ['margem_EBIT', False],
        'margem_liq': ['margem_liq', False],
        'margem_bruta': ['margem_bruta', False],
        'EV_EBITDA': ['EV_EBITDA', True],
        'EV_EBIT': ['EV_EBIT', True],
        'liq_corrente': ['liq_corrente', False],
        'liq_imediata': ['liq_imediata', False],
        'receitas_5anos': ['receitas_5anos', False],
        'lucros_5anos': ['lucros_5anos', False],
        'divida_liq_EBIT': ['divida_liq_EBIT', False],
        'divida_liq_patrimonio': ['divida_liq_patrimonio', False],
        'patrimonio_ativos': ['patrimonio_ativos', False],
        'passivos_ativos': ['passivos_ativos', False],
        'giro_ativos': ['giro_ativos', False],
        'liq_media_diaria': ['liq_media_diaria', False]
    }

    lista_duplas = []
    for indicador in lista_indicadores:
        lista_duplas.append(DICT_ASC[indicador])
    
    #df_indicadores = pd.read_csv(ARQUIVO_DADOS)
    df_indicadores = filtrar_indicadores(lista_indicadores)
    
    df_final = calcular_ranking(df_indicadores, lista_duplas, ano)

    df_final['posi_normalizada'] = round((100*df_final['soma_posi'])/(len(df_final)*len(indicadores)),2)

    colunas_certas = list(df_final.columns)
    colunas_certas.remove('ano')
    colunas_certas.remove('soma_posi')
    colunas_certas.remove('posi_normalizada')
    colunas_certas = ['soma_posi', 'posi_normalizada'] + colunas_certas

    return df_final[colunas_certas].copy()


In [23]:
def analisar_anos_anteriores(indicadores, tamanho_carteira, ano_inicial):
    ''' Analisar a melhor carteira de tamanho (tamanho_carteira) ao longo dos ultimos anos
    Recebe:
    dict - 'indicador': [min, max]
    int - quantidade de ativos
    int - ano inicial

    Retorna:
    dict - 'ano': [ativo1, ativo2, ..., ativoN]
    '''
    ativos_por_ano = {}

    for ano in range(ano_inicial, 2021, 1):
        df_ano_atual = analisar_indicadores(indicadores, ano)
        df_ano_atual = df_ano_atual.head(10).copy()
        #melhores_empresas = list(df_ano_atual['tick'])
        melhores_empresas = list(df_ano_atual.index)
        ativos_por_ano[str(ano)] = melhores_empresas
    
    for ano in ativos_por_ano:
        ativos_por_ano[ano] = ativos_por_ano[ano] + ([''] * (tamanho_carteira - len(ativos_por_ano[ano])))
    
    return ativos_por_ano


In [24]:
def filtrar_indicadores(dict_indicadores):
    df_indicadores = pd.read_csv(ARQUIVO_DADOS)

    for indicador in dict_indicadores:
        minimo, maximo = dict_indicadores[indicador]
        df_indicadores = df_indicadores[(df_indicadores[indicador] >= minimo) & (df_indicadores[indicador] <= maximo)].copy()

    return df_indicadores.copy()

In [25]:
indicadores = {'P_L': [0, 20], 'liq_media_diaria': [1000000, 20000000]}
#indicadores = {'P_L': [1, 10], 'liq_corrente': [1,2.5]}
ano = 2020

In [26]:
df_filtrado = filtrar_indicadores(indicadores)
df_filtrado

,tick,P_L,ano,margem_liq,divida_liq_EBITDA,liq_corrente,liq_imediata,LPA,VPA,P_VP,P_EBIT,P_EBITDA,P_ativos,margem_EBITDA,PSR,ROE,ROA,ROIC,EV_EBIT,EV_EBITDA,DY,preco,margem_bruta,margem_EBIT,divida_liq_EBIT,divida_liq_patrimonio,P_cap_giro,P_cirulante_liq,patrimonio_ativos,passivos_ativos,giro_ativos,receitas_5anos,lucros_5anos,liq_media_diaria
1857,VULC3,9.96,2020,9.69,0.06,4.78,0.70,0.51,4.49,1.14,10.55,6.87,0.91,0.14,0.97,11.40,9.10,9.76,10.07,6.54,0.00,5.10,34.41,9.14,-0.48,-0.05,1.92,-2.24,0.80,0.20,0.94,5.57,0.00,6022544.17
1861,SHUL4,10.24,2020,8.91,1.05,2.24,0.98,1.06,7.20,1.50,9.79,7.32,0.61,0.12,0.91,14.63,5.92,6.36,7.94,9.14,1.42,10.80,22.22,9.32,2.33,0.36,1.71,-1.69,0.40,0.60,0.66,7.65,12.44,2441440.87
1875,JPSA3,8.56,2020,32.74,0.87,2.70,2.37,2.75,34.38,0.68,4.60,3.61,0.34,0.79,2.80,7.99,4.01,6.19,8.36,6.44,0.00,23.50,64.99,60.86,3.75,0.56,3.07,-0.42,0.50,0.50,0.12,5.51,0.00,10537548.93
1878,ROMI3,9.04,2020,10.29,0.57,1.70,0.33,1.32,12.65,0.95,14.30,9.51,0.49,0.11,0.93,10.48,5.41,3.87,19.70,12.01,9.58,12.00,28.79,6.51,5.41,0.36,2.09,-1.14,0.52,0.48,0.53,3.37,76.10,2723393.53
1888,TRIS3,12.59,2020,19.73,0.27,3.18,0.90,0.83,6.03,1.74,11.43,11.37,1.05,0.22,2.48,13.81,8.33,10.83,10.84,10.70,3.06,10.50,36.88,21.73,-0.59,-0.09,2.00,-4.46,0.60,0.40,0.42,16.63,31.43,18807766.30
1890,PTBL3,12.00,2020,4.47,2.28,1.26,0.43,0.33,2.34,1.66,5.76,4.14,0.30,0.13,0.54,13.84,2.52,8.46,10.62,7.64,1.02,3.89,33.25,9.31,4.86,1.40,3.09,-0.57,0.18,0.82,0.56,3.26,-32.43,6176585.10
1906,LEVE3,11.50,2020,8.60,0.57,1.56,0.50,1.67,10.37,1.84,7.80,5.93,0.87,0.17,0.99,16.00,7.60,13.65,8.68,6.62,3.34,19.09,25.56,12.68,0.88,0.21,4.52,-1.90,0.47,0.53,0.88,1.60,5.40,9569506.60
1910,DIRR3,15.95,2020,7.98,0.69,4.24,1.46,0.76,9.40,1.28,9.81,8.21,0.37,0.15,1.27,8.04,2.32,6.41,10.67,8.75,4.30,11.62,34.36,12.97,0.86,0.11,1.01,-0.71,0.29,0.71,0.29,-4.50,-12.25,14111313.47
1922,GRND3,15.44,2020,22.19,0.09,5.14,2.72,0.50,3.99,1.92,19.24,15.62,1.65,0.22,3.43,12.45,10.66,8.44,16.29,13.27,3.45,7.67,45.81,17.80,-2.95,-0.29,3.28,-4.37,0.86,0.14,0.48,-1.50,0.40,15052171.20
1926,COCE5,8.99,2020,8.61,0.84,1.04,0.10,6.19,41.14,1.35,6.56,4.64,0.47,0.17,0.77,15.00,5.28,9.43,0.00,7.22,3.83,55.50,15.79,11.79,3.72,0.76,48.14,-0.65,0.35,0.65,0.61,8.25,9.99,2190700.73


In [27]:
df_ano_selecionado = analisar_indicadores(indicadores, ano)
df_ano_selecionado.head(10)

,soma_posi,posi_normalizada,rank_P_L
tick,,,
ALUP11,1.0,2.94,1.0
CLSC4,2.0,5.88,2.0
WIZS3,3.0,8.82,3.0
JPSA3,4.0,11.76,4.0
FESA4,5.0,14.71,5.0
COCE5,6.0,17.65,6.0
ROMI3,7.0,20.59,7.0
VULC3,8.0,23.53,8.0
SHUL4,9.0,26.47,9.0


In [28]:
ano_inicial = 2015
tamanho_carteira = 10
melhores_por_ano = analisar_anos_anteriores(indicadores, tamanho_carteira, ano_inicial)
#pprint(melhores_por_ano)
df_melhores_por_ano = pd.DataFrame(melhores_por_ano)
df_melhores_por_ano

,2015,2016,2017,2018,2019,2020
0,,,,,,ALUP11
1,,,,,,CLSC4
2,,,,,,WIZS3
3,,,,,,JPSA3
4,,,,,,FESA4
5,,,,,,COCE5
6,,,,,,ROMI3
7,,,,,,VULC3
8,,,,,,SHUL4
9,,,,,,LEVE3
